In [ ]:
pm_OrderDate = ""

In [ ]:
path_to_data = f'abfss://datalakehouse@synapsedp203dl.dfs.core.windows.net/refined/Customers/OrderDate={pm_OrderDate}'
customer_src_df = spark.read.format("delta").load(path_to_data)
display(customer_src_df)

In [ ]:
from delta.tables import *

deltaTableCustomers = DeltaTable.forPath(spark, 'abfss://datalakehouse@synapsedp203dl.dfs.core.windows.net/curated/Customers')

deltaTableCustomers.alias("tgt") \
  .merge(customer_src_df.alias("src"), "src.FirstName = tgt.FirstName AND src.LastName = tgt.LastName")\
  .whenMatchedUpdateAll()\
  .whenNotMatchedInsertAll()\
  .execute()

In [ ]:
path_to_data = f'abfss://datalakehouse@synapsedp203dl.dfs.core.windows.net/refined/ProductSales/OrderDate={pm_OrderDate}'
product_src_df = spark.read.format("delta").load(path_to_data)
display(product_src_df)

In [ ]:
spark.conf.set("spark.sql.sources.partitionOverwriteMode","dynamic")
product_src_df.write.mode("overwrite").partitionBy("OrderDate").format("delta").save(f'abfss://datalakehouse@synapsedp203dl.dfs.core.windows.net/curated/ProductSales')


In [ ]:
%%sql

select * from salesdb.productsales where OrderDate = '2024-02-22'